In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader
from ast import literal_eval

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# cifar10 dataset

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

# The batch size
batch_size = 512 

transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_set = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True,
    transform=transform_train,
)
test_set = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True,
    transform=transform_test
)

trains_set, validation_set = random_split(train_set, [0.75, 0.25])

train_loader = DataLoader(
    train_set, batch_size=batch_size, shuffle=True,
    num_workers=2
)
test_loader = DataLoader(
    test_set, batch_size=batch_size, shuffle=True,
    num_workers=2
)
validation_loader = DataLoader(
    validation_set, batch_size=batch_size, shuffle=True,
    num_workers=2
)

In [ ]:
class CNN(nn.Module):
    def __init__(self, conv1_hidden=6, conv2_hidden=16, conv_kernel_size=5, pool_kernel_size=2, pool_stride=2, linear1_hidden=120, linear2_hidden=84):
        super().__init__()
        self.conv1 = nn.Conv2d(3, conv1_hidden, conv_kernel_size)
        self.pool = nn.MaxPool2d(pool_kernel_size, pool_stride)
        self.conv2 = nn.Conv2d(conv1_hidden, conv2_hidden, conv_kernel_size)
        self.fc1 = nn.Linear(conv2_hidden * 5 * 5, linear1_hidden)
        self.fc2 = nn.Linear(linear1_hidden, linear2_hidden)
        self.fc3 = nn.Linear(linear2_hidden, 10)

    def forward(self, x):
        intermediate_outputs = []
        x = self.conv1(x)
        intermediate_outputs.append(x)
        x = self.pool(F.relu(x))
        intermediate_outputs.append(x)
        x = self.conv2(x)
        intermediate_outputs.append(x)
        x = self.pool(F.relu(x))
        intermediate_outputs.append(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        intermediate_outputs.append(x)
        x = F.relu(self.fc2(x))
        intermediate_outputs.append(x)
        x = F.softmax(self.fc3(x))
        intermediate_outputs.append(x)

        return x, intermediate_outputs
    
cnn = CNN().to(device)
optimizer_cnn = torch.optim.Adam(cnn.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(20):
    for data in train_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer_cnn.zero_grad()
        outputs, intermediate_outputs = cnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_cnn.step()

In [ ]:
# test to write in file
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs, intermediate_outputs = cnn(inputs)
        break

In [ ]:
labels = labels.detach().numpy()
outputs = outputs.detach().numpy()

# write in file
f = open("labels.csv", "w")
f.write("label;output\n")

for i in range(len(labels)):
    labels_str = labels[i]
    outputs_str = str(outputs[i])
    # remove \n from string
    outputs_str = outputs_str.replace("\n", "")
    f.write(str(labels_str) +";"+ str(outputs_str) +"\n")